# Cuantización

## Conceptos Clave

La **cuantización** es el proceso de convertir una señal de amplitud continua en una señal con un número finito de niveles discretos. Cuando una señal muestreada $x(n)$ se cuantiza, se obtiene una señal cuantizada $x_Q(n)$. La diferencia entre la señal cuantizada y la señal original muestreada es el **error de cuantización**, $e_q(n)$:
$$e_q(n) = x_Q(n) - x(n)$$

Si asumimos que el error de cuantización está uniformemente distribuido en el intervalo $(-\Delta/2, \Delta/2]$, donde $\Delta$ es el **tamaño del escalón de cuantización** (o paso de cuantización), el valor cuadrático medio (RMS) del error de cuantización es:
$$e_{rms} = \sqrt{E[e_q^2(n)]} = \frac{\Delta}{\sqrt{12}}$$

La relación entre el **rango dinámico de la señal** $R$ (diferencia entre el valor máximo y mínimo de la señal que el cuantizador puede representar) y el número de bits $B$ del cuantizador se establece a través del número de niveles de cuantización, $L = 2^B$:
$$R \approx (2^B) \Delta$$
Aunque a veces se usa $R = (2^B - 1) \Delta$ si $R$ es la distancia entre el nivel más bajo y el más alto. Para un número de bits $B$ suficientemente grande, la aproximación $R \approx 2^B \Delta$ es común. A partir de esta aproximación, podemos despejar el tamaño del escalón:
$$\Delta = \frac{R}{2^B}$$

El **Signal-to-Quantization Noise Ratio (SQNR)** es una medida de la calidad de la señal cuantizada. Se define como la relación entre la potencia de la señal y la potencia del ruido de cuantización, y a menudo se expresa en decibelios (dB):
$$SQNR_{dB} = 10 \log_{10} \left( \frac{P_x}{P_{e_q}} \right)$$
Donde $P_x$ es la potencia de la señal $x(n)$ y $P_{e_q}$ es la potencia del error de cuantización $e_q(n)$ (igual a $e_{rms}^2$ si el error tiene media cero).

Para una señal de entrada sinusoidal de amplitud pico $A$ (que ocupa todo el rango $R=2A$), con potencia $P_x = A^2/2$, y asumiendo que la potencia del error es $P_{e_q} = e_{rms}^2 = \Delta^2/12$, el SQNR teórico aproximado en función del número de bits $B$ es:
$$SQNR_{dB} \approx 6.02 B + 1.76$$
Una regla general común es que cada bit adicional en la cuantización mejora el SQNR en aproximadamente 6 dB.

## Ejemplo 1: Cálculo de Bits para una Señal con Rango y Error Específicos

Para determinar el número de bits de cuantización $B$ necesarios para una señal con un rango dinámico $R$ dado y un error RMS de cuantización $e_{rms}$ máximo permitido.

**Problema:**
Se tiene una señal con un rango dinámico de $R = 10 \text{ V}$ (por ejemplo, la señal varía de -5V a +5V). Se desea que el error RMS de cuantización sea menor o igual a $e_{rms} = 50 \mu V = 50 \times 10^{-6} \text{ V}$.
¿Cuántos bits de cuantización $B$ se necesitan? ¿Cuál es el SQNR teórico para este número de bits?

**Fórmulas a utilizar:**
1.  Relación entre $e_{rms}$ y $\Delta$:
    $e_{rms} = \frac{\Delta}{\sqrt{12}} \implies \Delta = e_{rms} \sqrt{12}$
2.  Relación entre $R$, $B$ y $\Delta$:
    $\Delta = \frac{R}{2^B} \implies 2^B = \frac{R}{\Delta}$
    Sustituyendo $\Delta$ de la primera fórmula:
    $2^B = \frac{R}{e_{rms} \sqrt{12}}$
3.  Despejando $B$:
    $B = \log_2 \left( \frac{R}{e_{rms} \sqrt{12}} \right)$
    Dado que $B$ debe ser un número entero (no podemos tener una fracción de bit), se tomará el valor techo ($\lceil B \rceil$) del resultado para asegurar que el $e_{rms}$ sea *menor o igual* al deseado.
4.  SQNR teórico aproximado:
    $SQNR_{dB} \approx 6.02 B + 1.76$

### Código Python para Ejemplo 1

In [3]:
# --------------- Imports ---------------
import numpy as np
import math

# --- Ejemplo 1: Cálculo de Bits y SQNR ---
print("--- Ejemplo 1: Cálculo de Bits y SQNR ---")

# Parámetros dados
R: float = 10.0  # Rango de la señal en Volts
erms_desired: float = 50e-6 # Error RMS de cuantización deseado (50 microVolts)

print(f"Rango de la señal (R): {R:.1f} V")
print(f"Error RMS deseado (erms_deseado): {erms_desired*1e6:.2f} uV")

# Calcular Delta basado en el error RMS deseado
# Delta = erms * sqrt(12)
Delta_calculated: float = erms_desired * math.sqrt(12)
print(f"Tamaño del escalón de cuantización requerido (Delta_calculado): {Delta_calculated:.6f} V")

# Calcular el número de niveles (2^B)
# R = 2^B * Delta  => 2^B = R / Delta
levels_float: float = R / Delta_calculated
print(f"Número de niveles (aproximado 2^B): {levels_float:.2f}")

# Calcular el número de bits (B)
# B = log2(R / Delta)
# Usamos math.ceil para asegurar que el error sea MENOR o igual al deseado,
# lo que significa que podríamos necesitar más bits para redondear hacia arriba.
B_calculated: int = math.ceil(math.log2(levels_float))
print(f"Número de bits de cuantización requeridos (B_calculado): {B_calculated}")

# Calcular el tamaño del escalón real y el error RMS real con el número de bits B_calculated
Delta_actual: float = R / (2**B_calculated)
erms_actual: float = Delta_actual / math.sqrt(12)
print(f"Tamaño del escalón real con {B_calculated} bits (Delta_real): {Delta_actual:.6f} V")
print(f"Error RMS real con {B_calculated} bits (erms_real): {erms_actual*1e6:.2f} uV (debería ser <= {erms_desired*1e6:.2f} uV)")

# Calcular el SQNR teórico (aproximado 6.02B + 1.76 dB)
sqnr_theoretical_db: float = 6.02 * B_calculated + 1.76
print(f"SQNR teórico con {B_calculated} bits: {sqnr_theoretical_db:.2f} dB")

print("\n--- Fin del Ejemplo 1 ---\n")

--- Ejemplo 1: Cálculo de Bits y SQNR ---
Rango de la señal (R): 10.0 V
Error RMS deseado (erms_deseado): 50.00 uV
Tamaño del escalón de cuantización requerido (Delta_calculado): 0.000173 V
Número de niveles (aproximado 2^B): 57735.03
Número de bits de cuantización requeridos (B_calculado): 16
Tamaño del escalón real con 16 bits (Delta_real): 0.000153 V
Error RMS real con 16 bits (erms_real): 44.05 uV (debería ser <= 50.00 uV)
SQNR teórico con 16 bits: 98.08 dB

--- Fin del Ejemplo 1 ---



## Ejemplo Extra: Visualización Interactiva del Efecto de la Cuantización

Este ejemplo te permite explorar interactivamente cómo la cuantización afecta una señal sinusoidal al variar el número de bits $B$. Podrás observar la señal original, la señal cuantizada y el error de cuantización, y cómo la forma del error y el SQNR cambian con $B$.

La cuantización implementada aquí divide el rango de la señal en $2^B$ intervalos (o escalones) uniformes. Cada muestra de la señal original se asigna al valor que representa el **centro del intervalo** en el que cae. Estos valores centrales son los niveles de salida discretos de la señal cuantizada.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import IntSlider, VBox, interactive_output, Layout
from IPython.display import display, clear_output
import math

# --- Constantes y Generación de Señal de Ejemplo ---
FS: int = 100  # Frecuencia de muestreo (Hz)
T_MAX: float = 1.0  # Duración de la señal (s)
t_vector: np.ndarray = np.arange(0, T_MAX, 1 / FS)

# Señal compuesta para la demostración
original_signal: np.ndarray = (
    0.7 * np.sin(2 * np.pi * 5 * t_vector) +
    0.3 * np.sin(2 * np.pi * 15 * t_vector + np.pi / 4) +
    0.05 * np.random.randn(len(t_vector))
)

signal_minimum_value: float = np.min(original_signal)
signal_peak_to_peak: float = np.max(original_signal) - signal_minimum_value

# Epsilon para evitar divisiones por cero y comparaciones numéricamente inestables
EPSILON: float = 1e-15

# --- Función de Cuantización ---
def quantize_signal_improved(
    signal: np.ndarray,
    bits: int,
    sig_range: float,
    sig_min: float
) -> tuple[np.ndarray, np.ndarray, float]:
    """
    Cuantiza una señal analógica a un número específico de bits.

    Args:
        signal (np.ndarray): La señal de entrada.
        bits (int): Número de bits para la cuantización.
        sig_range (float): Rango pico a pico de la señal (max - min).
        sig_min (float): Valor mínimo de la señal.

    Returns:
        tuple[np.ndarray, np.ndarray, float]:
            - signal_quantized: La señal cuantizada.
            - quantization_error: El error de cuantización (cuantizada - original).
            - sqnr_db: La relación señal a ruido de cuantización en dB.
    """
    if bits < 1:  # Caso especial para B=0 (1 solo nivel de cuantización)
        quantized_level: float = sig_min + sig_range / 2.0
        signal_quantized: np.ndarray = np.full_like(signal, quantized_level)
        quantization_error: np.ndarray = signal_quantized - signal

        signal_dc_offset: float = np.mean(signal)
        power_signal_ac: float = np.mean((signal - signal_dc_offset)**2) # Potencia AC
        power_error: float = np.mean(quantization_error**2)

        if power_error <= EPSILON:
            sqnr_db = np.inf if power_signal_ac > EPSILON else 0
        elif power_signal_ac <= EPSILON:
            sqnr_db = -np.inf if power_error > EPSILON else 0
        else:
            sqnr_db = 10 * np.log10(power_signal_ac / power_error)
        return signal_quantized, quantization_error, sqnr_db

    num_levels: int = 2**bits
    delta_step: float = sig_range / num_levels # Tamaño del escalón de cuantización

    # Manejo de señal con rango cero (DC) o delta_step muy pequeño
    if sig_range <= EPSILON or delta_step <= EPSILON:
        mean_val = sig_min + sig_range / 2.0
        signal_quantized = np.full_like(signal, mean_val)
        quantization_error = signal_quantized - signal
        
        power_signal_ac = np.mean((signal - np.mean(signal))**2)
        power_error = np.mean(quantization_error**2)

        if power_error <= EPSILON: sqnr_db = np.inf
        elif power_signal_ac <= EPSILON: sqnr_db = -np.inf
        else: sqnr_db = 10 * np.log10(power_signal_ac / power_error)
        return signal_quantized, quantization_error, sqnr_db

    # Bordes de los bins de cuantización (num_levels + 1 bordes para num_levels bins)
    quantization_edges: np.ndarray = sig_min + delta_step * np.arange(num_levels + 1)
    # Asegurar que el último borde cubra el máximo para robustez numérica
    quantization_edges[-1] = sig_min + sig_range

    # Asignar cada muestra al índice del bin (0-based)
    quantized_indices: np.ndarray = np.digitize(signal, quantization_edges) - 1
    # Clip para asegurar que los índices estén en [0, num_levels - 1]
    quantized_indices = np.clip(quantized_indices, 0, num_levels - 1)

    # Valor cuantizado es el centro del bin correspondiente
    signal_quantized: np.ndarray = sig_min + (quantized_indices * delta_step) + (delta_step / 2.0)
    quantization_error: np.ndarray = signal_quantized - signal

    # Cálculo del SQNR numérico
    signal_dc_offset: float = np.mean(signal)
    power_signal_ac: float = np.mean((signal - signal_dc_offset)**2)
    power_error: float = np.mean(quantization_error**2)

    if power_error <= EPSILON:
        sqnr_db = np.inf # Error de cuantización despreciable
    elif power_signal_ac <= EPSILON:
        sqnr_db = -np.inf # Señal de entrada principalmente DC
    else:
        sqnr_db = 10 * np.log10(power_signal_ac / power_error)
    
    return signal_quantized, quantization_error, sqnr_db


# --- Función de Actualización para el Widget Interactivo ---
# Este widget Output es el "lienzo" donde se dibujarán los gráficos actualizados.
output_plot_area = widgets.Output()

def update_quantization_plot_improved(B_bits: int): # El nombre del argumento (B_bits) DEBE COINCIDIR con la clave en interactive_output
    """Actualiza los gráficos al cambiar el número de bits."""
    # 'with output_plot_area:' dirige toda la salida de esta sección (incl. plt.show())
    # al widget output_plot_area.
    with output_plot_area:
        # clear_output limpia el contenido anterior del output_plot_area.
        # wait=True previene el parpadeo esperando que el nuevo contenido esté listo.
        clear_output(wait=True)

        quantized_signal, q_error, num_sqnr_db = quantize_signal_improved(
            original_signal, B_bits, signal_peak_to_peak, signal_minimum_value
        )

        fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=True)
        fig.suptitle(f'Efecto de la Cuantización (B = {B_bits} bits)', fontsize=14)

        axes[0].plot(t_vector, original_signal, label='Original', alpha=0.7, color='blue')
        # 'steps-post' ayuda a visualizar mejor los niveles discretos
        axes[0].plot(t_vector, quantized_signal, label='Cuantizada', alpha=0.9, color='orange', drawstyle='steps-post')
        axes[0].set_ylabel('Amplitud (V)')
        axes[0].set_title('Señal Original vs. Cuantizada')
        axes[0].legend(loc='upper right')
        axes[0].grid(True, linestyle='--', alpha=0.6)
        y_margin = signal_peak_to_peak * 0.1 if signal_peak_to_peak > EPSILON else 0.1
        axes[0].set_ylim(signal_minimum_value - y_margin, signal_minimum_value + signal_peak_to_peak + y_margin)

        axes[1].plot(t_vector, q_error, color='red', alpha=0.85)
        axes[1].set_ylabel('Error de Cuantización (V)')
        axes[1].set_title('Error de Cuantización')
        axes[1].grid(True, linestyle='--', alpha=0.6)
        if B_bits >= 1 and signal_peak_to_peak > EPSILON:
            current_delta: float = signal_peak_to_peak / (2**B_bits)
            max_theoretical_error: float = current_delta / 2.0
            axes[1].set_ylim(-max_theoretical_error * 1.5, max_theoretical_error * 1.5)
        elif signal_peak_to_peak <= EPSILON: # Señal DC o sin rango apreciable
            axes[1].set_ylim(-0.1 * (abs(signal_minimum_value) if abs(signal_minimum_value) > EPSILON else 1.0), 
                             0.1 * (abs(signal_minimum_value) if abs(signal_minimum_value) > EPSILON else 1.0))
        else: # Caso B=0
            axes[1].set_ylim(-signal_peak_to_peak * 0.6, signal_peak_to_peak * 0.6)

        axes[2].set_xlabel(f'Tiempo (s) - Muestreo a {FS} Hz')
        axes[2].set_ylabel('SQNR (dB)')
        axes[2].grid(True, linestyle='--', alpha=0.6)
        sqnr_display_text: str
        if np.isinf(num_sqnr_db):
            if num_sqnr_db > 0:
                sqnr_display_text = 'SQNR: Infinito dB (Error cero o despreciable)'
                axes[2].text(0.5, 0.5, sqnr_display_text, ha='center', va='center', transform=axes[2].transAxes, color='green', fontsize=11)
                axes[2].set_ylim(0, 120) # Rango fijo para SQNR Infinito
            else: # -np.inf
                sqnr_display_text = 'SQNR: -Infinito dB (Señal sin potencia AC o error muy grande)'
                axes[2].text(0.5, 0.5, sqnr_display_text, ha='center', va='center', transform=axes[2].transAxes, color='darkred', fontsize=11)
                axes[2].set_ylim(-80, 20) # Rango fijo para SQNR -Infinito
        else: # SQNR es un número finito
            sqnr_display_text = f'SQNR Numérico: {num_sqnr_db:.2f} dB'
            axes[2].axhline(num_sqnr_db, color='green', linestyle='-.', label=sqnr_display_text)
            axes[2].legend(loc='upper right')
            plot_min_y = min(num_sqnr_db - 15, -20 if B_bits > 0 else -70)
            plot_max_y = max(num_sqnr_db + 15, 30)
            axes[2].set_ylim(plot_min_y, plot_max_y)
        axes[2].set_title('Relación Señal a Ruido de Cuantización (SQNR)')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

# --- Configuración del Widget de Control (Slider) ---
slider_style = {'description_width': 'initial'} # Para que la descripción larga no se corte
slider_layout = Layout(width='70%', margin='0px 0px 10px 0px')

bits_slider = IntSlider(
    min=0,
    max=16, # Rango de bits (0 a 16 es un rango común para ADCs)
    step=1,
    value=3, # Valor inicial
    description='Número de Bits de Cuantización (B):',
    style=slider_style,
    layout=slider_layout,
    continuous_update=False # El gráfico se actualiza solo cuando se suelta el slider (más eficiente)
)

# --- Conexión del Widget con la Función de Ploteo ---
# interactive_output vincula el cambio en 'bits_slider' con el argumento 'B_bits'
# de la función 'update_quantization_plot_improved'.
interactive_plot_object = interactive_output(
    update_quantization_plot_improved,
    {'B_bits': bits_slider}
)

# --- Despliegue de la Interfaz de Usuario ---
print("Ajusta el slider para cambiar el número de bits de cuantización:")
# Es crucial mostrar tanto el slider como el output_plot_area donde se dibujarán los gráficos.
display(VBox([bits_slider, output_plot_area]))

# Ejecutar la función de ploteo una vez inicialmente para mostrar el estado por defecto.
# Esto también debe ocurrir dentro del contexto de output_plot_area.
with output_plot_area:
    update_quantization_plot_improved(bits_slider.value)

print("\n--- Fin del Código Interactivo ---")

Ajusta el slider para cambiar el número de bits de cuantización:



--- Fin del Código Interactivo ---
